<a href="https://colab.research.google.com/github/mnarasim/mle9_capstone/blob/develop/spotify_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import numpy as np
import sklearn
import torch
import torchaudio
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler
import skimage.io
from PIL import Image
from google.colab import drive 
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [3]:
'''
gauth = GoogleAuth()   
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)  
# read false samples
false_dict = {'name':[], 'id':[]}
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format('13yBGR5nWOgp_U-cbYSi5XZZ0Sr1JxQlj')}).GetList()
for file in file_list:

    false_dict['name'].append(file['title'])
    false_dict['id'].append(file['id'])
   

    
# read true samples
true_dict = {'name':[], 'id':[]}
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format('1mUhbkiEhDEAZgfpoO85k9yNwnGDdmLR4')}).GetList()
for file in file_list:

    true_dict['name'].append(file['title'])
    true_dict['id'].append(file['id'])

'''
DRIVE_FOLDER = '/content/drive/My Drive/data/false/'
entries = [entry for entry in os.listdir(DRIVE_FOLDER) if os.path.isfile(os.path.join(DRIVE_FOLDER, entry))]
false_dict = {'name':[]}
for file in entries:

    false_dict['name'].append(file)
DRIVE_FOLDER = '/content/drive/My Drive/data/true/'
entries = [entry for entry in os.listdir(DRIVE_FOLDER) if os.path.isfile(os.path.join(DRIVE_FOLDER, entry))]
true_dict = {'name':[]}
for file in entries:

    true_dict['name'].append(file)

In [4]:
df1 = pd.DataFrame(false_dict)
df1['label'] = 0
df1['path'] = '/content/drive/My Drive/data/false/'

df2 = pd.DataFrame(true_dict)
df2['label'] = 1
df2['path'] = '/content/drive/My Drive/data/true/'
# combine both dataframes
df = pd.concat([df1,df2])
df = df.sample(frac=1, random_state=56).reset_index(drop=True)

print('The length of the dataset is: ', f'{df.shape[0]:,}')

labels = df['label'].unique()
df.head

The length of the dataset is:  1,375


<bound method NDFrame.head of                    name  label                                 path
0      Mani_false56.wav      0  /content/drive/My Drive/data/false/
1     Mani_false438.wav      0  /content/drive/My Drive/data/false/
2     Mani_false473.wav      0  /content/drive/My Drive/data/false/
3      Mani_false39.wav      0  /content/drive/My Drive/data/false/
4           Mani235.wav      1   /content/drive/My Drive/data/true/
...                 ...    ...                                  ...
1370   Maninoise651.wav      1   /content/drive/My Drive/data/true/
1371    Maninoise39.wav      1   /content/drive/My Drive/data/true/
1372        Mani694.wav      1   /content/drive/My Drive/data/true/
1373  Mani_false319.wav      0  /content/drive/My Drive/data/false/
1374  Mani_false395.wav      0  /content/drive/My Drive/data/false/

[1375 rows x 3 columns]>

In [5]:
#Split the dataset into train and test dataset
trainset, testset = train_test_split(df, test_size=0.25, random_state=100)

#read_file = drive.CreateFile({'id': trainset.iloc[0][1]})
#read_file.GetContentFile(trainset.iloc[0][0])

waveform = torchaudio.load(trainset.iloc[0][2] + trainset.iloc[0][0])[0]
sr = torchaudio.load(trainset.iloc[0][2] + trainset.iloc[0][0])[1]

In [6]:
#load data required 
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        
    
        
        waveform = torchaudio.load(row['path'] + row["name"])[0]

   
        return (
            waveform, row["label"],row['name'], row['path']
        )

train = MyDataset(trainset)
test = MyDataset(testset)



In [7]:
new_sample_rate = 16000
transform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=new_sample_rate)
transformed = transform(waveform)

In [8]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
  
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)


def collate_fn(batch):

    # A data tuple has the form:
    # waveform, label

    tensors, targets, name, ids = [], [], [], []

    # Gather in lists, and encode labels as indices
    for waveform, label, n, i in batch:
        tensors += [waveform]
        label = torch.tensor(label)
    
        targets += [label]
        name.append(n)
        ids.append(i)

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)
    #name = torch.stack(name)

    return tensors, targets, name, ids


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 250

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    train,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
test_loader = torch.utils.data.DataLoader(
    test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

cuda


In [10]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)


model = M5(n_input=transformed.shape[0], n_output=len(labels))
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)

M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=2, bias=True)
)
Number

In [11]:
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10

In [12]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target,name, paths) in enumerate(train_loader):

        
        data = data.to(device)
        target = target.to(device)
        name = target.to(device)
        paths = target.to(device)
        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = F.nll_loss(output.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [15]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)

def correct_ones_name(pred, target, name):
  
    pred = pred.squeeze()

    
    mask = torch.eq(pred, target)
    mask = mask.tolist()

    result = [x for x, y in zip(name, mask) if y == True]
 
    return result

def correct_ones_path(pred, target, paths):
  
    pred = pred.squeeze()

    
    mask = torch.eq(pred, target)
    mask = mask.tolist()

    result = [x for x, y in zip(paths, mask) if y == True]
 
    return result

def test(model, epoch):
    model.eval()
    correct = 0
    correct_tensors=[]
    
    d = '/content/drive/My Drive/data/results/'
    entries = [entry for entry in os.listdir(d) if os.path.isfile(os.path.join(d, entry))]
    for f in entries:
      try:
        os.remove(d + f)
      except OSError:
        pass


    i=0
    for data, target,name, paths in test_loader:
        i = i + 1
  
        
        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        pred = get_likely_index(output)
       
        correct_name = correct_ones_name(pred, target,name)
        correct_path = correct_ones_path(pred, target,paths)
       
      
        df2 = pd.DataFrame(data = correct_name, columns=['name'])
        df3 = pd.DataFrame(data = correct_path, columns=['path'])
        df4 = pd.concat([df2, df3], axis=1)
        
        p = d+ 'data'+str(i)+'.csv'
        #df4.to_csv('data'+str(i)+'.csv')
        with open(p, 'w', encoding = 'utf-8-sig') as f:
          df4.to_csv(f)
     

        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")
    

 
  


In [16]:
log_interval = 20
n_epoch = 25

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        test(model, epoch)
        scheduler.step()

# Let's plot the training loss versus the number of iteration.
# plt.plot(losses);
# plt.title("training loss");

  0%|          | 0/25 [00:00<?, ?it/s]

Train Epoch: 1 [0/1031 (0%)]	Loss: 0.331231

Test Epoch: 1	Accuracy: 205/344 (60%)

Train Epoch: 2 [0/1031 (0%)]	Loss: 0.274833

Test Epoch: 2	Accuracy: 276/344 (80%)

Train Epoch: 3 [0/1031 (0%)]	Loss: 0.140926

Test Epoch: 3	Accuracy: 294/344 (85%)

Train Epoch: 4 [0/1031 (0%)]	Loss: 0.109798

Test Epoch: 4	Accuracy: 297/344 (86%)

Train Epoch: 5 [0/1031 (0%)]	Loss: 0.054793

Test Epoch: 5	Accuracy: 286/344 (83%)

Train Epoch: 6 [0/1031 (0%)]	Loss: 0.045714

Test Epoch: 6	Accuracy: 325/344 (94%)

Train Epoch: 7 [0/1031 (0%)]	Loss: 0.105642

Test Epoch: 7	Accuracy: 332/344 (97%)

Train Epoch: 8 [0/1031 (0%)]	Loss: 0.080342

Test Epoch: 8	Accuracy: 329/344 (96%)

Train Epoch: 9 [0/1031 (0%)]	Loss: 0.091670

Test Epoch: 9	Accuracy: 335/344 (97%)

Train Epoch: 10 [0/1031 (0%)]	Loss: 0.077798

Test Epoch: 10	Accuracy: 338/344 (98%)

Train Epoch: 11 [0/1031 (0%)]	Loss: 0.044731

Test Epoch: 11	Accuracy: 339/344 (99%)

Train Epoch: 12 [0/1031 (0%)]	Loss: 0.039405

Test Epoch: 12	Accuracy: 3

In [17]:
d = '/content/drive/My Drive/data/results/'
df_res1 = pd.read_csv(d+'data1.csv')
df_res2 = pd.read_csv(d+'data2.csv')
df_final = pd.concat([df_res1, df_res2])
df_final.head(4)


,Unnamed: 0,name,path
0,0,Mani_false50.wav,/content/drive/My Drive/data/false/
1,1,Mani233.wav,/content/drive/My Drive/data/true/
2,2,Mani_false307.wav,/content/drive/My Drive/data/false/
3,3,Mani306.wav,/content/drive/My Drive/data/true/


In [26]:

import speech_recognition as sr

for i in range(df_final.shape[0]):

     
     name_audio = df_final.iloc[i][1]

     print(name_audio)
    
     r = sr.Recognizer()
     with sr.WavFile(df_final.iloc[i][2] + name_audio) as source:              
        audio = r.record(source)                        

        try:
            print("Transcription: " + r.recognize_google(audio))   
        except Exception as e:
            print("Error: " + str(e))

Mani_false50.wav
Transcription: they could barely see the boy
Mani233.wav
Transcription: weed killer do you know what he likes his food plane gifts for 7 year
Mani_false307.wav
Transcription: all you've got to do is talk to him
Mani306.wav
Transcription: 1/4 brain what should I watch
Mani292 (1).wav
Transcription: Seafood brain how does Amazon deploy recommendation system
Maninoise47.wav
Transcription: Fayetteville train when did Michael Jordan retire from basketball
Mani_false74.wav
Transcription: I'll be back in a minute
Mani_false506.wav
Transcription: it left a greenish streak that glowed for some seconds
Mani_false140.wav
Transcription: he's talking like a child
Mani_false299.wav
Transcription: sadly my dream of becoming a squirrel Whisperer may never happen
Mani_false286.wav
Transcription: it's required an explanation I'm just as sweet Universe meets as it travels through endless time
Mani351 (1).wav
Transcription: u.s. open starts on August 29th 11:30 when does it
Maninoise83.wa